# Router.parallel_acompletions

In [ ]:
import os, asyncio
from litellm import Router
router = Router(default_litellm_params={
  'api_base': os.environ['CHUTES_API_BASE'],
  'api_key': None,
  'custom_llm_provider': 'openai_like',
  'extra_headers': {'x-api-key': os.environ['CHUTES_API_KEY'], 'Authorization': os.environ['CHUTES_API_KEY']},
})
prompts = ['Say OK-A','Say OK-B','Say OK-C']
reqs = [{'model': os.environ['CHUTES_MODEL'], 'messages':[{'role':'user','content':p}], 'kwargs': {'max_tokens': 8, 'temperature': 0}} for p in prompts]
async def run():
  out = await router.parallel_acompletions(requests=reqs, concurrency=3)
  print([o.get('choices',[{}])[0].get('message',{}).get('content','') for o in out])
asyncio.run(run())